BoxPlots for All patients

In [1]:
#Import Modules

import sys
sys.path.append("..") #give this script access to all modules in parent directory
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from scipy.io import loadmat
import scipy
from scipy.stats import skew
from Data_Cleaning import preprocess
from Data_Cleaning import preprocess_sickbay
import Filtering
import Actigraph_Metrics

C:\Users\jakes\AppData\Local\Temp\ipykernel_27840\888196890.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Define Functions

Load All Patient Data

In [3]:
#data_dir = 'C:/Users/sidha/OneDrive/Sid Stuff/PROJECTS/iMEDS Design Team/Data Analysis/PedAccel/data_analysis/PythonPipeline/PatientData'
# data_dir = r'C:\Users\sidha\OneDrive\Sid Stuff\PROJECTS\iMEDS Design Team\Data Analysis\PedAccel\data_analysis\PythonPipeline\PatientData'

data_dir = r'C:\Users\jakes\Documents\DT 6 Analysis\PythonCode\PedAccel\data_analysis\PythonPipeline\PatientData'
window_size = 100 #100 is 1 second worth of time
lead_time = 10
slice_size_min = 15
sr = .5


In [12]:
def fillArrays(signal, Mean, AUC, PEAK_PEAK): 
    x = []
    for i in range(len(signal)):
        x.append(i) #x is the x axis for signal data, it is necessary for AUC calculations. Signal is ordered as is x, so this works
    Mean.append(np.mean(signal))
    AUC.append(Actigraph_Metrics.calc_area_under_curve(x,signal))
    PEAK_PEAK.append(max(signal)-min(signal))

def fill_SBS_Changes(vitals_SBS,SBS_Changes,i):
    state = vitals_SBS[i+1]-vitals_SBS[i]
    if(state > 0):
        SBS_Changes.append(1) #1 signifies increase in SBS
    elif state == 0: 
        SBS_Changes.append(0)
    else:
        SBS_Changes.append(0)

# def fill_Changes(vitals_SBS,SBS_Changes,i,Mean,Mean_Changes,AUC,AUC_Changes,PEAK_PEAK,PEAK_PEAK_Changes):
#     fill_SBS_Changes(vitals_SBS,SBS_Changes,i)
#     #Populate Mean_Changes
#     count_mean = 0
#     count_peak = 0
#     count_AUC = 0

#     state = Mean[i+1] - Mean[i]
#     if(state > 0):
#         Mean_Changes.append(1)
#     else: 
#         Mean_Changes.append(-1)
#     if(Mean_Changes[i] == SBS_Changes[i]):
#         count_mean+=1

#     state1 = PEAK_PEAK[i+1] - PEAK_PEAK[i]
#     if(state1 > 0):
#         PEAK_PEAK_Changes.append(1)
#     else: 
#         PEAK_PEAK_Changes.append(-1)
#     if(PEAK_PEAK_Changes[i] == SBS_Changes[i]):
#         count_peak+=1

#     state2 = AUC[i+1] - AUC[i]
#     if(state2 > 0):
#         AUC_Changes.append(1)
#     else: 
#         AUC_Changes.append(-1)
#     if(AUC_Changes[i] == SBS_Changes[i]):
#         count_AUC+=1

In [31]:

#There is no error handling in place, the .mat file must exist
for patient in os.listdir(data_dir):
    # filter out non-directories
    patient_dir = os.path.join(data_dir, patient)
    if os.path.isdir(patient_dir):
       # data_filepath_accel = os.path.join(patient_dir, f'{patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')           
        data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY_{slice_size_min - lead_time}MIN_{lead_time}MIN.mat')
        
       # accel_data = loadmat(data_filepath_accel)
       # x_mag = accel_data["x_mag"]
       # accel_SBS = accel_data["sbs"].flatten()
        
        vitals_data = loadmat(data_filepath_vitals)
        temp_hr = vitals_data['heart_rate']
        temp_SpO2 = vitals_data['SpO2']
        temp_rr = vitals_data['respiratory_rate']
        temp_bps = vitals_data['blood_pressure_systolic']
        temp_bpm = vitals_data['blood_pressure_mean']
        temp_bpd = vitals_data['blood_pressure_diastolic']
        vitals_SBS = vitals_data['sbs'].flatten()
        hr = []
        rr = []
        SpO2 = []
        bpm = []
        bps = []
        bpd = []
        vitals_list = [hr,rr,SpO2,bpm,bps,bpd]
        vitals_names = ['hr','rr','spo2','bpm','bps','bpd']
        temp_vitals = [temp_hr,temp_rr, temp_SpO2,temp_bpm,temp_bps,temp_bpd] 
        
        flag_list = [0] * (int)(sr * 60 * slice_size_min) #generate a list to insert in place of invalid data, 
        #this list serves as a flag for a window to ignore in the box plot function

        for j in range(len(vitals_list)): #go through every vitals metric
            print(f'original {vitals_names[j]} vitals array shape: {np.array(temp_vitals[j]).shape} ')
            for i in range(len(vitals_SBS)): #go through every SBS score for each vitals metric
                if (Filtering.checkVitals(temp_vitals[j][i], slice_size_min, vitals_names[j])): #check the data in a single window
                    vitals_list[j].append(temp_vitals[j][i]) #append that single window data to the 2D hr,rr,spo2,bpm,bps,bpd arrays if that window's data is valid
                else:
                    vitals_list[j].append(flag_list) #append an array of zeros for window number i for the jth vitals metric if the data is invalid(i.e. too many NaN points)
                    print(f'{vitals_names[j]} SBS index {i} has insufficient data, zeros appended in place') 
            print(f'final {vitals_names[j]} vitals array shape: {np.array(vitals_list[j]).shape}') #should be the number of SBS scores by the number of samples in a window


        j = 0

        for k in  range(len(vitals_list)): 
            
            SBS_Changes = []
            AUC_Changes = []
            PEAK_PEAK_Changes = []
            Mean_Changes = []
            cleared_SBS = []
            AUC = []
            PEAK_PEAK = []
            Mean = []

            print(f'Calculating {vitals_names[k]} information for {patient}\n')
            signal_2D = (np.array(vitals_list[k])) #signal_2D is a single vitals metric

            for j in range(len(signal_2D)):
                if(np.mean(signal_2D[j])!=0): #check that neither is a flag list, signal_2D[j] is a single window for a single metric
                #define arrays to populate on each iteration
                    # Populate array with changes in SBS
                    cleared_SBS.append(vitals_SBS[j])
                    fillArrays(signal_2D[j], Mean, AUC, PEAK_PEAK)
                else:
                    print(f'flag list detected at index {j}')

            i = 0
            print(cleared_SBS)
            
            assert(len(Mean)== len(AUC))
            assert(len(AUC) == len(PEAK_PEAK))
            assert(len(cleared_SBS)==len(Mean))
            assert(len(vitals_SBS) >= len(cleared_SBS))

            count_mean = 0
            count_peak = 0
            count_AUC = 0
    
            factor = (1/20)

            if(len(cleared_SBS) > 3):
                q3, q1 = np.percentile(np.array(Mean), [75 ,25])
                iqr = q3 - q1
                threshold = factor * iqr
                q3, q1 = np.percentile(np.array(PEAK_PEAK), [75 ,25])
                iqr = q3 - q1
                threshold1 = factor * iqr
                q3, q1 = np.percentile(np.array(AUC), [75 ,25])
                iqr = q3 - q1
                threshold2 = factor * iqr     

                for i in range(len(cleared_SBS)-1):
                    fill_SBS_Changes(cleared_SBS,SBS_Changes,i)

                    state = Mean[i+1] - Mean[i]
                    if(state > threshold):
                        Mean_Changes.append(1)
                    elif(state < -1*threshold): 
                        Mean_Changes.append(-1)
                    else:
                        Mean_Changes.append(0)
                    if(Mean_Changes[i] == SBS_Changes[i]):
                        count_mean+=1

                    state1 = PEAK_PEAK[i+1] - PEAK_PEAK[i]
                    if(state1 > threshold1):
                        PEAK_PEAK_Changes.append(1)
                    elif(state1< -1*threshold1): 
                        PEAK_PEAK_Changes.append(-1)
                    else:
                        PEAK_PEAK_Changes.append(0)
                    if(PEAK_PEAK_Changes[i] == SBS_Changes[i]):
                        count_peak+=1

                    state2 = AUC[i+1] - AUC[i]
                    if(state2 > threshold2):
                        AUC_Changes.append(1)
                    elif state2 < -1*threshold2: 
                        AUC_Changes.append(-1)
                    else:
                        AUC_Changes.append(0)
                    if(AUC_Changes[i] == SBS_Changes[i]):
                        count_AUC+=1

                print(f'The below stats are based on {len(cleared_SBS)} SBS scores\n')
                print(f"Percentage of same changes with AUC is: {100*(count_AUC/len(SBS_Changes))}%\n")
                print(f"Percentage of same changes with Peak to Peak is: {100*(count_peak/len(SBS_Changes))}%\n")
                print(f"Percentage of same changes with Mean is: {100*(count_mean/len(SBS_Changes))}%\n")
            else:
                print('not enough SBS scores')




original hr vitals array shape: (14, 450) 
final hr vitals array shape: (14, 450)
original rr vitals array shape: (14, 450) 
rr SBS index 0 has insufficient data, zeros appended in place
rr SBS index 1 has insufficient data, zeros appended in place
rr SBS index 2 has insufficient data, zeros appended in place
rr SBS index 3 has insufficient data, zeros appended in place
rr SBS index 4 has insufficient data, zeros appended in place
rr SBS index 5 has insufficient data, zeros appended in place
rr SBS index 6 has insufficient data, zeros appended in place
rr SBS index 7 has insufficient data, zeros appended in place
rr SBS index 8 has insufficient data, zeros appended in place
rr SBS index 9 has insufficient data, zeros appended in place
rr SBS index 10 has insufficient data, zeros appended in place
rr SBS index 11 has insufficient data, zeros appended in place
rr SBS index 12 has insufficient data, zeros appended in place
rr SBS index 13 has insufficient data, zeros appended in place
fin